In [1]:
import numpy as np
import nibabel as nib
import nilearn
from nilearn.image import resample_to_img
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.plotting import show
from nilearn.plotting import plot_roi
from nilearn import image
from nilearn.masking import apply_mask
# get_ipython().magic('matplotlib inline')
import scipy
import matplotlib
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.input_data import NiftiMasker
#from nilearn import plotting
import nibabel
from nilearn.masking import apply_mask
from nilearn.image import load_img
from nilearn.image import new_img_like
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, svm, metrics
from sklearn.linear_model import Ridge
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import KFold
from sklearn.cross_validation import LeaveOneLabelOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_selection import SelectFwe
from scipy import signal
from scipy.fftpack import fft, fftshift
from scipy import interp

params = {'legend.fontsize': 'large',
          'figure.figsize': (5, 3),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
font = {'weight': 'bold',
        'size': 22}
plt.rc('font', **font)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, f_classif, GenericUnivariateSelect, SelectKBest, chi2
from sklearn.feature_selection import RFE
import os
import seaborn as sns
import pandas as pd
import csv
from scipy import stats
import brainiak
import brainiak.funcalign.srm


/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
projectDir='/jukebox/norman/amennen/prettymouth_fmriprep2/'
DMNmask='/jukebox/norman/amennen/MNI_things/Yeo_JNeurophysiol11_MNI152/Yeo_Network7mask_reoriented_resampledBOLD2.nii.gz'
fmriprep_dir=projectDir + '/derivatives/fmriprep'
preproc_dir = projectDir + '/derivatives/preproc'
TOM_large=projectDir + 'ROI/TOM_large_resampled_maskedbybrain.nii.gz'
TOM_cluster=projectDir + 'ROI/TOM_cluster_resampled_maskedbybrain.nii.gz'

In [3]:
# load subject numbers
subInd = 0
nsub=38
allnames = []
allgroups = []
groupInfo={}
# skip subjects 039 and 116
with open(projectDir + 'participants.tsv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        if 'sub' in row[0]:
            # now skip the subjects we don't want to analyze
            allInfo = row[0].split('\t')
            subjName=allInfo[0]
            if subjName != 'sub-039' and subjName != 'sub-116':
                if allInfo[3] == 'paranoia':
                    group = 0
                elif allInfo[3] == 'affair':
                    group = 1
                allnames.append(subjName)
                allgroups.append(group)
                subInd+=1

In [4]:
paranoidSubj = allnames[0:19]
cheatingSubj = allnames[19:]
paranoidLabel = allgroups[0:19]
cheatingLabel = allgroups[19:]
nfolds=19
print(allnames)

['sub-088', 'sub-089', 'sub-090', 'sub-091', 'sub-092', 'sub-093', 'sub-094', 'sub-095', 'sub-096', 'sub-097', 'sub-098', 'sub-099', 'sub-100', 'sub-101', 'sub-102', 'sub-103', 'sub-104', 'sub-105', 'sub-106', 'sub-107', 'sub-108', 'sub-109', 'sub-110', 'sub-111', 'sub-068', 'sub-081', 'sub-112', 'sub-053', 'sub-113', 'sub-031', 'sub-114', 'sub-115', 'sub-117', 'sub-118', 'sub-119', 'sub-120', 'sub-121', 'sub-122']


In [8]:
# first load in data, remove first 2 TRs, resave
n_trunc=2 # Number of volumes to trim/truncate

nSub = 38
for s in np.arange(nSub):
    print(s)
    subjName=allnames[s]
    print(subjName)
    subjData=fmriprep_dir + '/' + subjName + '/' + 'func' + '/' + subjName + '_task-prettymouth_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    epi_data = nib.load(subjData)
    epi=epi_data.get_fdata()
    epi_trunc =np.zeros((epi_data.shape[0], epi_data.shape[1], epi_data.shape[2], epi_data.shape[3]-n_trunc))
    epi_trunc[:, :, :, :] = epi[:,:,:,n_trunc:]
    #print(epi_data.shape, '  ', epi_trunc.shape)
    dimsize=epi_data.header.get_zooms()
    #print(dimsize)
    affine_mat = epi_data.affine  # What is the orientation of the data
    #print(affine_mat)
    # now save
    output_name = preproc_dir + '/' + subjName + '/' + 'func' + '/' + subjName + '_task-prettymouth_space-MNI152NLin2009cAsym_desc-preproc_bold_truncTRs.nii.gz'
    bold_nii = nib.Nifti1Image(epi_trunc, affine_mat)
    hdr = bold_nii.header  # get a handle for the .nii file's header
    hdr.set_zooms((dimsize[0], dimsize[1], dimsize[2], epi_trunc.shape[3]))
    if not os.path.exists(os.path.dirname(output_name)):
        os.makedirs(os.path.dirname(output_name))
    nib.save(bold_nii, output_name)

0
sub-088
1
sub-089
2
sub-090
3
sub-091
4
sub-092
5
sub-093
6
sub-094
7
sub-095
8
sub-096
9
sub-097
10
sub-098
11
sub-099
12
sub-100
13
sub-101
14
sub-102
15
sub-103
16
sub-104
17
sub-105
18
sub-106
19
sub-107
20
sub-108
21
sub-109
22
sub-110
23
sub-111
24
sub-068
25
sub-081
26
sub-112
27
sub-053
28
sub-113
29
sub-031
30
sub-114
31
sub-115
32
sub-117
33
sub-118
34
sub-119
35
sub-120
36
sub-121
37
sub-122


In [14]:
# now load in the new truncated and highpass filter with two different versions
n_trunc=2 # Number of volumes to trim/truncate
nSub = 38

filterY = 337.75 # Yaara's filter
filterS = 140 # Sam's filter
# make masker
##### FOR DMN
#nVox=3757
#### FOR TOM_Large
#nVox=2414
#### FOR CLUSTERS
nVox = 240
nTR = 475 - n_trunc
aggregate_masked_trunc_filt_Y = np.zeros((nVox,nTR,nSub))
aggregate_masked_trunc_filt_S = np.zeros((nVox,nTR,nSub))
for s in np.arange(nSub):
    subjName=allnames[s]
    print(subjName)
    epi_file = preproc_dir + '/' + subjName + '/' + 'func' + '/' + subjName + '_task-prettymouth_space-MNI152NLin2009cAsym_desc-preproc_bold_truncTRs.nii.gz'

    epi_masker = NiftiMasker(mask_img = TOM_cluster,
                             high_pass=1/filterY,
                             standardize=False,
                             t_r=1.5,
                             memory='nilearn_cache',
                             memory_level=1,
                             verbose=0
                            )
    epi_mask_dataY = epi_masker.fit_transform(epi_file)
    aggregate_masked_trunc_filt_Y[:,:,s] = epi_mask_dataY.T

    epi_masker = NiftiMasker(mask_img = TOM_cluster,
                             high_pass=1/filterS,
                             standardize=False,
                             t_r=1.5,
                             memory='nilearn_cache',
                             memory_level=1,
                             verbose=0
                            )
    epi_mask_dataS = epi_masker.fit_transform(epi_file)
    aggregate_masked_trunc_filt_S[:,:,s] = epi_mask_dataS.T


sub-088
sub-089
sub-090
sub-091
sub-092
sub-093
sub-094
sub-095
sub-096
sub-097
sub-098
sub-099
sub-100
sub-101
sub-102
sub-103
sub-104
sub-105
sub-106
sub-107
sub-108
sub-109
sub-110
sub-111
sub-068
sub-081
sub-112
sub-053
sub-113
sub-031
sub-114
sub-115
sub-117
sub-118
sub-119
sub-120
sub-121
sub-122


In [15]:
# now save the two different aggregate data types
#np.save('aggregate_data_trunc_filtY', aggregate_masked_trunc_filt_Y)
#np.save('aggregate_data_trunc_filtS', aggregate_masked_trunc_filt_S)
#np.save('aggregate_data_TOM_large_trunc_filtY', aggregate_masked_trunc_filt_Y)
#np.save('aggregate_data_TOM_large_trunc_filtS', aggregate_masked_trunc_filt_S)
np.save('aggregate_data_TOM_cluster_trunc_filtY', aggregate_masked_trunc_filt_Y)
np.save('aggregate_data_TOM_cluster_trunc_filtS', aggregate_masked_trunc_filt_S)

In [3]:
import numpy as np
np.arange(34)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [7]:
# DO THIS FOR THE DATA THAT ISN'T TRUNCATED/FILTERED****
#nVox = 3757 DMN mask
#nVox = 2414 # TOM_large
nVox = 240 # TOM clusster
nTRs = 475
nSub = 38
aggregate_masked_data = np.zeros((nVox,nTRs,nSub))
for s in np.arange(nsub):
    subjName=allnames[s]
    print(subjName)
    subjData=fmriprep_dir + '/' + subjName + '/' + 'func' + '/' + subjName + '_task-prettymouth_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    #masked_data = apply_mask(subjData, DMNmask)
    masked_data = apply_mask(subjData,TOM_cluster) # CHANGE HERE TOO
    aggregate_masked_data[:,:,s] = masked_data.T

#np.save('aggregate_data.npy', aggregate_masked_data)
np.save('aggregate_data_TOM_cluster.npy', aggregate_masked_data)

sub-088
sub-089
sub-090
sub-091
sub-092
sub-093
sub-094
sub-095
sub-096
sub-097
sub-098
sub-099
sub-100
sub-101
sub-102
sub-103
sub-104
sub-105
sub-106
sub-107
sub-108
sub-109
sub-110
sub-111
sub-068
sub-081
sub-112
sub-053
sub-113
sub-031
sub-114
sub-115
sub-117
sub-118
sub-119
sub-120
sub-121
sub-122


In [9]:
np.save('aggregate_data_TOM_cluster.npy', aggregate_masked_data)